In [9]:
import json

  
def generateExplanation(label, targets, words, post_tokens):
    if label == 1: #normal
        return "None"
    
    string = "this sentence is "
    if label == 0: #hate speech
        string += "hate speech because it uses discriminatory, prejorative, and violent language"
    elif label == 2: #offensive
        string +="offensive because it contains insult and negative expression"
     
    if len(targets)!=0 and targets[0].lower() != "none":
        string += " against"
        if len(targets)==1:
            string += " " + targets[0]
        else:
            for i in range(len(targets)):
                if i == len(targets)-1 and len(targets)>1:
                    string = string[:-1]
                    string += " and "
                    string += targets[i] 
                    break

                string += " "
                string +=targets[i]+ ","
            
    string += " through words "
    
    #assume words contains at least 1 word

   # Process `words` to group consecutive indices into phrases

    grouped_words = []
    current_group = [words[0]]
    if len(words)==0:
        print(data)
    for i in range(1, len(words)):
        if words[i] == words[i - 1] + 1:  # Consecutive index
            current_group.append(words[i])
        else:
            # Add the current group as a phrase
            grouped_words.append(" ".join(post_tokens[idx] for idx in current_group))
            current_group = [words[i]]

    # Add the last group
    grouped_words.append(" ".join(post_tokens[idx] for idx in current_group))

    # Format grouped words with single quotes and "and" for the last item
    if len(grouped_words) == 1:
        string += f"'{grouped_words[0]}'"
    else:
        string += ", ".join(f"'{phrase}'" for phrase in grouped_words[:-1])
        string += f" and '{grouped_words[-1]}'"

    string += "."
    return string

    


In [10]:
easy_train_data_new = []
# Load the dataset from a JSON file
with open("easy_train_data.json", "r") as f:
    easy_train_data = json.load(f)
    
for data in easy_train_data:
    id = data["id"]
    label = data["label"]
    targets = data["target"]
    rationale_tokens = data["rationale"]
    post_tokens = data["post_tokens"]
    
    dict = {"0.5": [], "above": []}
    for i in range(len(rationale_tokens)):
        if id == "1179091366283862016_twitter" and rationale_tokens[i] >=0.3:
            dict["above"].append(i)
        if rationale_tokens[i] >0.5:
            dict["above"].append(i)
        elif rationale_tokens[i] == 0.5:
            dict["0.5"].append(i)
    if len(dict["above"]) >0:
        words = dict["above"]
    else:
        words = dict["0.5"]
            
    explanation_text = generateExplanation(label, targets, words, post_tokens)
    data["explanation"] = explanation_text
    easy_train_data_new.append(data)
 
# Save list as a JSON file
with open('e1_easy_train_data.json', 'w') as f:
    json.dump(easy_train_data_new, f)
    
# for data in easy_train_data_new:
#     if data["label"] == 1 or data["label"] == 0:
#         continue
    
#     print(data["id"])
#     print(" ".join(data["post_tokens"]))
#     print(data["explanation"])
#     print()

In [11]:
with open("easy_eval_data.json", "r") as f:
    easy_eval_data = json.load(f)
    
easy_eval_data_new = []

for data in easy_eval_data:
    id = data["id"]
    label = data["label"]
    targets = data["target"]
    rationale_tokens = data["rationale"]
    post_tokens = data["post_tokens"]
    
    dict = {"0.5": [], "above": []}
    for i in range(len(rationale_tokens)):
        if rationale_tokens[i] >0.5:
            dict["above"].append(i)
        elif rationale_tokens[i] == 0.5:
            dict["0.5"].append(i)
    if len(dict["above"]) >0:
        words = dict["above"]
    else:
        words = dict["0.5"]
            
    explanation_text = generateExplanation(label, targets, words, post_tokens)
    data["explanation"] = explanation_text
    easy_eval_data_new.append(data)
 
# Save list as a JSON file
with open('e1_easy_eval_data.json', 'w') as f:
    json.dump(easy_eval_data_new, f)